In [4]:
#Import all the dependies needed for the task 
import pandas as pd 
import matplotlib as plt
import scipy.stats as st 

#Show the files we will be using 
mouse_data_path = "Data/Mouse_metadata.csv"
study_results_path = "Data/Study_results.csv"

#Read the files allowing us to access them more freely
mouse_metadata = pd.read_csv(mouse_data_path)
study_results = pd.read_csv(study_results_path)

In [ ]:
#Combine the data into one dataset